# Notebook to extract and curate ChEMBL data for the Leeson data set (drug-target interactions)

### Authors: Barbara Zdrazil, Lina Heinzke
### 10/2022

**This notebook extracts data from ChEMBL and performs some curation steps in order to retrieve a data set for drug-target, and clinical candidate-target associations including comparator compounds.**

**The notebook is based on initial work by Anne Hersey, Patrica Bento, Emma Manners, Paul Leeson, and Andrew Leach.**

**More documentation on the initial data set compilation can be found here ("Ligand Efficiency"): https://www.ebi.ac.uk/seqdb/confluence/pages/viewpage.action?spaceKey=CHEMBL&title=Anne%27s+Notes**


In [1]:
import pandas as pd
import numpy as np

#### notebook settings
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

# Get data from ChEMBL

In [2]:
# # @Barbara: uncomment and modify this to your preferred paths
# path_results = "/Users/bzdrazil/Dropbox/ChEMBL/NP/data/"
# path_sqlite3_database = <your sqlite database location>

chembl_version = "26"
base_path = "/Users/heinzke/Documents/PhD/Projects/drug_target_dataset_curation/"
path_results = base_path+"results/"
path_sqlite3_database = base_path+"data/chembl_"+chembl_version+"/chembl_"+chembl_version+"_sqlite/chembl_"+chembl_version+".db"

In [3]:
# # @Barbara: Accessing ChEMBL using Oracle
# # If you want to use this option, you have to change the sql statements to the commented line:
# # from docs -> from chembl_31.docs

# import cx_Oracle

# #cx_Oracle.init_oracle_client(lib_dir="/Users/bzdrazil/Downloads/instantclient_19_8")  #https://www.oracle.com/in/database/technologies/instant-client/macos-intel-x86-downloads.html; https://stackoverflow.com/questions/56119490/cx-oracle-error-dpi-1047-cannot-locate-a-64-bit-oracle-client-library
# cx_Oracle.clientversion() 

# import pandas as pd
# import sqlalchemy as sa
# import requests

# workdir = '/Users/bzdrazil/Desktop/'

# chemdev2 = 'oracle://{}:{}@ora-dlvm-103.ebi.ac.uk:1521/?service_name=chemdev2'.format('user', 'pw') # insert your username and password
# engine_ch = sa.create_engine(chemdev2)

In [4]:
import sqlite3

engine_ch = sqlite3.connect(path_sqlite3_database)

In [5]:
sql = '''

select mh.parent_molregno, 
    docs.year, docs.journal, 
    act.pchembl_value, act.standard_type, 
    ass.assay_type, ass.tid, 
    md.chembl_id as compound_chembl_id, md.pref_name as compound_pref_name, md.max_phase, 
    md.first_approval, md.prodrug, md.oral, md.parenteral, md.topical, md.black_box_warning, 
    td.pref_name as Target_pref_name, td.target_type, td.organism, td.chembl_id as Target_chembl_id,
    vs.mutation
from docs, activities act, molecule_hierarchy mh, assays ass, target_dictionary td, molecule_dictionary md
-- from chembl_31.docs, chembl_31.activities act, chembl_31.molecule_hierarchy mh, chembl_31.assays ass, chembl_31.target_dictionary td, chembl_31.molecule_dictionary md
left join variant_sequences vs
    on vs.variant_id = ass.variant_id
where mh.molregno=act.molregno
    and act.pchembl_value is not null
    -- and ass.assay_type = 'B'
    and act.assay_id = ass.assay_id
    and act.doc_id = docs.doc_id
    and ass.tid = td.tid
    and md.molregno = mh.parent_molregno
    and act.potential_duplicate = 0
    and data_validity_comment is null
    and act.standard_relation = '='
    and td.tid <>22226   ----exclude unchecked targets
    and td.target_type like '%PROTEIN%'

'''

df_mols = pd.read_sql_query(sql, con=engine_ch)
df_mols['tid_mutation'] = np.where(df_mols['mutation'].notnull(), 
                                   df_mols['tid'].astype('str')+'-'+df_mols['mutation'], 
                                   df_mols['tid'].astype('str'))
df_mols

,parent_molregno,year,journal,pchembl_value,standard_type,assay_type,tid,compound_chembl_id,compound_pref_name,max_phase,first_approval,prodrug,oral,parenteral,topical,black_box_warning,Target_pref_name,target_type,organism,Target_chembl_id,mutation,tid_mutation
0,252199,2004.0,Bioorg. Med. Chem. Lett.,5.40,IC50,B,10483,CHEMBL357278,None,0,NaN,-1,0,0,0,0,Palmitoyl-CoA oxidase,SINGLE PROTEIN,Rattus norvegicus,CHEMBL4632,None,10483
1,253534,2004.0,Bioorg. Med. Chem. Lett.,4.77,IC50,B,10483,CHEMBL357119,None,0,NaN,-1,0,0,0,0,Palmitoyl-CoA oxidase,SINGLE PROTEIN,Rattus norvegicus,CHEMBL4632,None,10483
2,253199,2004.0,Bioorg. Med. Chem. Lett.,6.75,IC50,B,10483,CHEMBL152968,None,0,NaN,-1,0,0,0,0,Palmitoyl-CoA oxidase,SINGLE PROTEIN,Rattus norvegicus,CHEMBL4632,None,10483
3,253199,2004.0,Bioorg. Med. Chem. Lett.,5.22,IC50,A,12594,CHEMBL152968,None,0,NaN,-1,0,0,0,0,Cytochrome P450 1A2,SINGLE PROTEIN,Homo sapiens,CHEMBL3356,None,12594
4,253199,2004.0,Bioorg. Med. Chem. Lett.,4.43,IC50,A,17045,CHEMBL152968,None,0,NaN,-1,0,0,0,0,Cytochrome P450 3A4,SINGLE PROTEIN,Homo sapiens,CHEMBL340,None,17045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475921,2317531,2018.0,J Med Chem,8.10,Ki,B,11522,CHEMBL4278080,None,0,NaN,-1,0,0,0,0,Cholecystokinin B receptor,SINGLE PROTEIN,Homo sapiens,CHEMBL298,None,11522
2475922,2334349,2018.0,J Med Chem,9.22,Kd,B,134,CHEMBL4294901,None,0,NaN,-1,0,0,0,0,Vasopressin V1a receptor,SINGLE PROTEIN,Homo sapiens,CHEMBL1889,None,134
2475923,2329285,2018.0,J Med Chem,8.42,Ki,B,134,CHEMBL4289837,None,0,NaN,-1,0,0,0,0,Vasopressin V1a receptor,SINGLE PROTEIN,Homo sapiens,CHEMBL1889,None,134
2475924,2317951,2018.0,J Med Chem,7.89,Ki,B,134,CHEMBL4278500,None,0,NaN,-1,0,0,0,0,Vasopressin V1a receptor,SINGLE PROTEIN,Homo sapiens,CHEMBL1889,None,134


In [6]:
df_mols = df_mols.astype({
    'year': 'Int64',
    'first_approval': 'Int64'
})

In [7]:
# df_mols.to_csv(path_results+"ChEMBL"+chembl_version+"_initial_query.csv", sep = ';')

In [8]:
############### TESTING: load comparison data set ###############
def get_sets(data):
    comp_mols = set(data["PARENT_MOLREGNO"]) 
    comp_drugs = set(data[data['DTI'] == 'D_DT']["PARENT_MOLREGNO"])
    comp_targets = set(data["TID_mutation"]) 
    comp_drug_targets = set(data[data['DTI'] == 'D_DT']["TID_mutation"]) 
    comp_pairs = set(data["PAIRS"])
    comp_drug_pairs = set(data[data['DTI'] == 'D_DT']["PAIRS"])
    return comp_mols, comp_drugs, comp_targets, comp_drug_targets, comp_pairs, comp_drug_pairs

# full comparison dataset
df_comparison = pd.read_csv(base_path+"data/DTI_2021_final.dsv", sep = "\t")
df_comparison[['TID_no_ending', 'ending']] = df_comparison['TID'].str.split('-', 1, expand=True)
df_comparison['TID_mutation'] = np.where(((df_comparison['ending'].notnull()) & (df_comparison['ending'] != 'WT')), 
                                   df_comparison['TID_no_ending']+'-'+df_comparison['ending'], 
                                   df_comparison['TID_no_ending'])
df_comparison['PAIRS'] = df_comparison.agg('{0[PARENT_MOLREGNO]}_{0[TID_mutation]}'.format, axis=1)
df_comparison = df_comparison.astype({'TID_no_ending': 'Int64', 'PARENT_MOLREGNO': 'Int64'})


comp_mols, comp_drugs, comp_targets, comp_drug_targets, comp_pairs, comp_drug_pairs = get_sets(df_comparison)

comparison_list = ["comparison", 
                   len(comp_mols), len(comp_drugs), 
                   len(comp_targets), len(comp_drug_targets), len(comp_pairs), len(comp_drug_pairs)]
all_intersect_comp = [comparison_list]
all_length_comp = [comparison_list]

# limit the dataset to the targets that have at least 100 comparator compounds
min_nof_cpds = 100
comparator_cpds_100 = df_comparison[df_comparison['MAX_PHASE'] < 4].groupby(['TID'])['PARENT_MOLREGNO'].count()
targets_enough_cpds = comparator_cpds_100[comparator_cpds_100 >= min_nof_cpds].index.tolist()
df_comparison_100 = df_comparison.query('TID in @targets_enough_cpds')

comp_mols_100, comp_drugs_100, comp_targets_100, comp_drug_targets_100, comp_pairs_100, comp_drug_pairs_100 = get_sets(df_comparison_100)

comparison_list_100 = ["comparison_100", 
                       len(comp_mols_100), len(comp_drugs_100), 
                       len(comp_targets_100), len(comp_drug_targets_100), len(comp_pairs_100), len(comp_drug_pairs_100)]
all_intersect_comp_100 = [comparison_list_100]
all_length_comp.append(comparison_list_100)

In [9]:
############### TESTING: method to print comparison to original dataset ###############
def print_comparison(description, now, comp, output=False):
    intersect = now.intersection(comp)
    if output:
        print("------")
        print(description)
        print("{:15}".format("current:"), len(now))
        print("{:15}".format("comparison:"), len(comp))
        print("{:15}".format("intersection:"), len(intersect))
    return len(intersect)


def print_all_comparisons(now_mols, now_drugs, now_targets, now_drug_targets, now_pairs, now_drug_pairs, output=False, only_100=False):
    intersections = []
    descriptions = ["Mols (molregno)", "Drugs (molregno)", "Targets (tid)", "Drug targets (tid)", "Compound-target pairs",  "Drug-target pairs"]
    now_results = [now_mols, now_drugs, now_targets, now_drug_targets, now_pairs, now_drug_pairs]
    if only_100:
        comparison_results = [comp_mols_100, comp_drugs_100, comp_targets_100, comp_drug_targets_100, comp_pairs_100, comp_drug_pairs_100]
    else:
        comparison_results = [comp_mols, comp_drugs, comp_targets, comp_drug_targets, comp_pairs, comp_drug_pairs]
    
    for description, now, comp in zip(descriptions, now_results, comparison_results):
        intersections.append(print_comparison(description, now, comp, output))
    return intersections


def add_intersections(data, label, output=False):
    now_mols = set(data["parent_molregno"]) 
    now_drugs = set(data[data["max_phase"] == 4]["parent_molregno"]) 
    now_targets = set(data["tid_mutation"]) 
    now_drug_targets = set(data[data["max_phase"] == 4]["tid_mutation"]) 
    now_pairs = set(data.agg('{0[parent_molregno]}_{0[tid_mutation]}'.format, axis=1))
    now_drug_pairs = set(data[data["max_phase"] == 4].agg('{0[parent_molregno]}_{0[tid_mutation]}'.format, axis=1))
    
    comp = print_all_comparisons(now_mols, now_drugs, now_targets, now_drug_targets, now_pairs, now_drug_pairs, output, False)
    all_intersect_comp.append([label] + comp)
    all_length_comp.append([label, len(now_mols), len(now_drugs), len(now_targets), len(now_drug_targets), len(now_pairs), len(now_drug_pairs)])

    if output:
        print("..................... >= 100 cpds .....................")
    comp_100 = print_all_comparisons(now_mols, now_drugs, now_targets, now_drug_targets, now_pairs, now_drug_pairs, output, True)
    all_intersect_comp_100.append([label] + comp_100)

In [10]:
############### TESTING: initial query ###############
add_intersections(df_mols, "init", True)

------
Mols (molregno)
current:        923862
comparison:     378661
intersection:   378661
------
Drugs (molregno)
current:        1637
comparison:     686
intersection:   616
------
Targets (tid)
current:        6893
comparison:     910
intersection:   907
------
Drug targets (tid)
current:        2569
comparison:     351
intersection:   349
------
Compound-target pairs
current:        2094029
comparison:     561772
intersection:   560759
------
Drug-target pairs
current:        21601
comparison:     1221
intersection:   1120
..................... >= 100 cpds .....................
------
Mols (molregno)
current:        923862
comparison:     372275
intersection:   372275
------
Drugs (molregno)
current:        1637
comparison:     642
intersection:   572
------
Targets (tid)
current:        6893
comparison:     521
intersection:   520
------
Drug targets (tid)
current:        2569
comparison:     269
intersection:   267
------
Compound-target pairs
current:        2094029
comparison:

# Calculate mean, median, and max pChEMBL values for each compound-target pair

In [11]:
# summarise pchembl values into mean, max, median and year into first publication
df_mols['pchembl_value_mean'] = df_mols.groupby(['parent_molregno', 'tid_mutation'])['pchembl_value'].transform('mean')
df_mols['pchembl_value_max'] = df_mols.groupby(['parent_molregno', 'tid_mutation'])['pchembl_value'].transform('max')
df_mols['pchembl_value_median'] = df_mols.groupby(['parent_molregno', 'tid_mutation'])['pchembl_value'].transform('median')
df_mols['first_publication_target_cmpd_pair'] = df_mols.groupby(['parent_molregno', 'tid_mutation'])['year'].transform('min')

In [12]:
# repeat based on the data of only the binding assays
df_mols_binding = df_mols[df_mols['assay_type'] == 'B'].copy()
df_mols_binding['pchembl_value_mean'] = df_mols_binding.groupby(['parent_molregno', 'tid_mutation'])['pchembl_value'].transform('mean')
df_mols_binding['pchembl_value_max'] = df_mols_binding.groupby(['parent_molregno', 'tid_mutation'])['pchembl_value'].transform('max')
df_mols_binding['pchembl_value_median'] = df_mols_binding.groupby(['parent_molregno', 'tid_mutation'])['pchembl_value'].transform('median')
df_mols_binding['first_publication_target_cmpd_pair'] = df_mols_binding.groupby(['parent_molregno', 'tid_mutation'])['year'].transform('min')

In [13]:
# table that has rows for pchembl_mean, max, median based on all assay data (only_binding = False)
# and rows for values based on only binding assays (only_binding = True)
df_mols['only_binding'] = False
df_mols_binding['only_binding'] = True
df_mols_limited = pd.concat([df_mols, df_mols_binding])
df_mols_limited = df_mols_limited.drop(columns=['year', 'journal', 'pchembl_value', 'standard_type', 'assay_type'])
df_mols_limited = df_mols_limited.drop_duplicates()

# Add compound properties

In [14]:
sql_cpd_props = '''

select mh.parent_molregno, 
    cp.mw_freebase, cp.alogp, cp.hba, cp.hbd, cp.psa, cp.rtb, cp.ro3_pass, cp.num_ro5_violations, 
    cp.cx_most_apka, cp.cx_most_bpka, cp.cx_logp, cp.cx_logd, cp.molecular_species, cp.full_mwt, 
    cp.aromatic_rings, cp.heavy_atoms, cp.qed_weighted, cp.mw_monoisotopic, cp.full_molformula, 
    cp.hba_lipinski, cp.hbd_lipinski, cp.num_lipinski_ro5_violations, 
    struct.standard_inchi, struct.standard_inchi_key, struct.canonical_smiles
from compound_properties cp, molecule_hierarchy mh, compound_structures struct
-- from chembl_31.compound_properties cp, chembl_31.molecule_hierarchy mh, chembl_31.compound_structures struct
where cp.molregno=mh.parent_molregno
    and struct.molregno=mh.parent_molregno

'''

df_cpd_props = pd.read_sql_query(sql_cpd_props, con=engine_ch)
df_cpd_props.head()

,parent_molregno,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,cx_most_apka,cx_most_bpka,cx_logp,cx_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,standard_inchi,standard_inchi_key,canonical_smiles
0,1,341.75,2.11,5.0,1.0,84.82,3.0,N,0.0,6.48,NaN,3.63,2.69,ACID,341.75,3.0,24.0,0.74,341.0567,C17H12ClN3O3,6.0,1.0,0.0,InChI=1S/C17H12ClN3O3/c1-10-8-11(21-17(24)20-15(22)9-19-21)6-7-12(10)16(23)13-4-2-3-5-14(13)18/h...,OWRSAHYFSSNENM-UHFFFAOYSA-N,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl
1,2,332.32,1.33,6.0,1.0,108.61,3.0,N,0.0,6.33,NaN,2.88,1.82,ACID,332.32,3.0,25.0,0.73,332.0909,C18H12N4O3,7.0,1.0,0.0,InChI=1S/C18H12N4O3/c1-11-8-14(22-18(25)21-16(23)10-20-22)6-7-15(11)17(24)13-4-2-12(9-19)3-5-13/...,ZJYUMURGSZQFMH-UHFFFAOYSA-N,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1
2,3,357.80,2.27,5.0,2.0,87.98,3.0,N,0.0,6.33,NaN,3.70,2.64,ACID,357.80,3.0,25.0,0.75,357.0880,C18H16ClN3O3,6.0,2.0,0.0,InChI=1S/C18H16ClN3O3/c1-10-7-14(22-18(25)21-15(23)9-20-22)8-11(2)16(10)17(24)12-3-5-13(19)6-4-1...,YOMWDCALSDWFSV-UHFFFAOYSA-N,Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1
3,4,307.31,1.46,5.0,1.0,84.82,3.0,N,0.0,6.33,NaN,3.02,1.97,ACID,307.31,3.0,23.0,0.74,307.0957,C17H13N3O3,6.0,1.0,0.0,InChI=1S/C17H13N3O3/c1-11-2-4-12(5-3-11)16(22)13-6-8-14(9-7-13)20-17(23)19-15(21)10-18-20/h2-10H...,PSOPUAQFGCRDIP-UHFFFAOYSA-N,Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1
4,5,341.75,2.11,5.0,1.0,84.82,3.0,N,0.0,6.33,NaN,3.63,2.57,ACID,341.75,3.0,24.0,0.74,341.0567,C17H12ClN3O3,6.0,1.0,0.0,InChI=1S/C17H12ClN3O3/c1-10-8-13(21-17(24)20-15(22)9-19-21)6-7-14(10)16(23)11-2-4-12(18)5-3-11/h...,KEZNSCMBVRNOHO-UHFFFAOYSA-N,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1


In [15]:
df_cpd_props_unique = df_cpd_props.drop_duplicates()

In [16]:
# df_cpd_props_unique.to_csv(path_results+"ChEMBL"+chembl_version+"_cpd_props.csv", sep = ';')

## Combine initial query with compound properties

In [17]:
df_combined = df_mols_limited.merge(df_cpd_props_unique, on = 'parent_molregno', how = 'inner')

In [18]:
# df_combined.to_csv(path_results+"ChEMBL"+chembl_version+"_cpds_with_props.csv", sep = ';')

In [19]:
############### TESTING ###############
print("All mols:", df_mols_limited.shape)
print("Mols with available properties:", df_combined.shape)
print("Difference:", len(df_mols_limited) - len(df_combined))

All mols: (3160231, 22)
Mols with available properties: (3156881, 47)
Difference: 3350


In [20]:
############### TESTING: compound props ###############
add_intersections(df_mols_limited, "cpd props")

# Calculate LE metrics

Ligand Efficiency (LE):

$\text{LE} = \frac{\Delta\text{G}}{\text{HA}}$
where $ \Delta\text{G} = − RT \ln(K_d)$, $− RT\ln(K_i)$, or $− RT\ln(IC_{50})$

$\text{LE}=\frac{(2.303 \cdot 298 \cdot 0.00199 \cdot \text{pchembl_value})} {\text{heavy_atoms}}$


$\text{BEI}=\frac{\text{pchembl_mean} \cdot 1000} {\text{mw_freebase}}$

$\text{SEI}=\frac{\text{pchembl_mean} \cdot 100} {\text{PSA}}$

$\text{LLE}=\text{pchembl_mean}-\text{ALOGP}$

In [21]:
df_combined['LE'] = df_combined['pchembl_value_mean']/df_combined['heavy_atoms']*(2.303*298*0.00199)
df_combined['BEI'] = df_combined['pchembl_value_mean']*1000/df_combined["mw_freebase"]
df_combined['SEI'] = df_combined['pchembl_value_mean']*100/df_combined["psa"]
df_combined['LLE'] = df_combined['pchembl_value_mean']-df_combined["alogp"]

# Extract drug-target interactions with disease relevance from drug_mechanism table

In [22]:
sql_dti = '''
select distinct mh.parent_molregno, dm.tid, dm.disease_efficacy
from drug_mechanism dm, molecule_hierarchy mh, molecule_dictionary md
where mh.molregno=dm.molregno
and md.molregno=mh.parent_molregno
'''

df_dti = pd.read_sql_query(sql_dti, con=engine_ch)
# drop rows that don't have a tid
df_dti = df_dti.dropna(subset = ['tid'])
df_dti['tid'] = df_dti['tid'].astype('Int64')
df_dti

,parent_molregno,tid,disease_efficacy
0,1124,11060,1
1,675068,10193,1
2,1125,10193,1
3,1085,10193,1
4,1124,10193,1
...,...,...,...
4756,1304559,101019,1
4757,1304559,100417,1
4758,2336099,11540,1
4759,2146132,100097,1


In [23]:
# disease_efficacy NUMBER
# Flag to show whether the target assigned is believed to play a role in the efficacy of the drug in the indication(s)
# for which it is approved (1 = yes, 0 = no)
df_dti = df_dti[df_dti['disease_efficacy'] == 1].copy()

## Add single proteins for protein families, protein complexes, protein complex groups
## Query protein mappings

In [24]:
sql_dti = '''
select tr.tid, tr.relationship, tr.related_tid, 
    td1.pref_name as pref_name_1, td1.target_type as target_type_1, td1.organism as organism_1, 
    td2.pref_name as pref_name_2, td2.target_type as target_type_2, td2.organism as organism_2, td2.chembl_id as Target_chembl_id_2 
from target_relations tr
inner join target_dictionary td1
    on tr.tid = td1.tid
inner join target_dictionary td2
    on tr.related_tid = td2.tid
'''

df_related_targets = pd.read_sql_query(sql_dti, con=engine_ch)
df_related_targets.head()

,tid,relationship,related_tid,pref_name_1,target_type_1,organism_1,pref_name_2,target_type_2,organism_2,Target_chembl_id_2
0,10193,SUBSET OF,104764,Carbonic anhydrase I,SINGLE PROTEIN,Homo sapiens,Carbonic anhydrase,PROTEIN FAMILY,Homo sapiens,CHEMBL2095180
1,12071,SUBSET OF,109746,Cyclin-dependent kinase 1,SINGLE PROTEIN,Homo sapiens,Cyclin-dependent kinase,PROTEIN FAMILY,Homo sapiens,CHEMBL3559691
2,12071,SUBSET OF,104709,Cyclin-dependent kinase 1,SINGLE PROTEIN,Homo sapiens,Cyclin-dependent kinase 1/cyclin B,PROTEIN COMPLEX,Homo sapiens,CHEMBL2094127
3,12071,SUBSET OF,107893,Cyclin-dependent kinase 1,SINGLE PROTEIN,Homo sapiens,CDK1/Cyclin A,PROTEIN COMPLEX,Homo sapiens,CHEMBL3038467
4,12071,SUBSET OF,117095,Cyclin-dependent kinase 1,SINGLE PROTEIN,Homo sapiens,Cyclin-dependent kinase 1/G1/S-specific cyclin-D1,PROTEIN COMPLEX,Homo sapiens,CHEMBL3885551


In [25]:
protein_complex_mapping = df_related_targets[(df_related_targets["target_type_1"] == "PROTEIN COMPLEX") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "SUPERSET OF")]

protein_family_mapping = df_related_targets[(df_related_targets["target_type_1"] == "PROTEIN FAMILY") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "SUPERSET OF")]

protein_complex_group_mapping = df_related_targets[(df_related_targets["target_type_1"] == "PROTEIN COMPLEX GROUP") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "SUPERSET OF")]

# TODO: should these be included? which direction (how to avoid duplications)?
single_protein_mapping = df_related_targets[(df_related_targets["target_type_1"] == "SINGLE PROTEIN") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "EQUIVALENT TO")]

# TODO: should these be included?
chimeric_protein_mapping = df_related_targets[(df_related_targets["target_type_1"] == "CHIMERIC PROTEIN") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "SUPERSET OF")]

# TODO: should these be included?
ppi_mapping = df_related_targets[(df_related_targets["target_type_1"] == "PROTEIN-PROTEIN INTERACTION") 
                    & (df_related_targets["target_type_2"] == "SINGLE PROTEIN")
                    & (df_related_targets["relationship"] == "SUPERSET OF")]

In [26]:
relevant_mappings = pd.concat([protein_complex_mapping, 
                               protein_family_mapping, 
                               protein_complex_group_mapping,
                               #single_protein_mapping, 
                               #chimeric_protein_mapping, 
                               #ppi_mapping, 
                               ])
relevant_mappings['tid'] = relevant_mappings['tid'].astype('Int64')
relevant_mappings.head()

,tid,relationship,related_tid,pref_name_1,target_type_1,organism_1,pref_name_2,target_type_2,organism_2,Target_chembl_id_2
249,104282,SUPERSET OF,10819,Acetylcholine receptor; alpha1/beta1/delta/gamma,PROTEIN COMPLEX,Homo sapiens,Acetylcholine receptor protein alpha chain,SINGLE PROTEIN,Homo sapiens,CHEMBL4808
251,104282,SUPERSET OF,12715,Acetylcholine receptor; alpha1/beta1/delta/gamma,PROTEIN COMPLEX,Homo sapiens,Acetylcholine receptor protein delta chain,SINGLE PROTEIN,Homo sapiens,CHEMBL3011
260,104284,SUPERSET OF,10023,Neuronal acetylcholine receptor; alpha2/beta4,PROTEIN COMPLEX,Rattus norvegicus,Neuronal acetylcholine receptor protein alpha-2 subunit,SINGLE PROTEIN,Rattus norvegicus,CHEMBL2584
261,104284,SUPERSET OF,12717,Neuronal acetylcholine receptor; alpha2/beta4,PROTEIN COMPLEX,Rattus norvegicus,Neuronal acetylcholine receptor subunit beta-4,SINGLE PROTEIN,Rattus norvegicus,CHEMBL2658
275,104685,SUPERSET OF,49,Integrin alpha-IIb/beta-3,PROTEIN COMPLEX,Homo sapiens,Integrin alpha-IIb,SINGLE PROTEIN,Homo sapiens,CHEMBL212


In [27]:
# DT-interactions and targets based on drug_mechanisms table
DTIs_original = set(df_dti.agg('{0[parent_molregno]}_{0[tid]}'.format, axis=1))
dti_tids_original = set(df_dti['tid'])

# DT-interactions and targets with other target IDs based on mapping
df_dti_add_targets = df_dti.merge(relevant_mappings, on = 'tid', how = 'inner')
DTIs_mapped = set(df_dti_add_targets.agg('{0[parent_molregno]}_{0[related_tid]}'.format, axis=1))
dti_tids_mapped = set(df_dti_add_targets['related_tid'].astype("int"))

# combined DT-interactions and targets
DTIs_set = DTIs_original.union(DTIs_mapped)
dti_tids_set = dti_tids_original.union(dti_tids_mapped)

In [28]:
############### TESTING: drug_mechanisms available data comparison ###############
# Including mapped target IDs significantly improves the intersection 
# between the comparison dataset and the drug_mechanisms table.
# However, there are generally targets and drug-target pairs not present in the drug_mechanisms table
# that are present in the old dataset.
df_comparison['PAIRS_tid'] = df_comparison.agg('{0[PARENT_MOLREGNO]}_{0[TID_no_ending]}'.format, axis=1)
comp_dti_targets = set(df_comparison["TID_no_ending"]) 
comp_dti_drug_pairs = set(df_comparison[df_comparison['DTI'] == 'D_DT']["PAIRS_tid"])

dti_mols = set(df_dti['parent_molregno'])
mol_intersect = print_comparison("Drugs (molregno)", dti_mols, comp_drugs, output=True)

print('................. Before mapping target IDs .................')
dti_targets = dti_tids_original
targets_intersect = print_comparison("Targets (tid)", dti_targets, comp_dti_targets, output=True)

dti_pairs = DTIs_original 
pairs_intersect = print_comparison("Drug-target pairs", dti_pairs, comp_dti_drug_pairs, output=True)

print('................. Including mapping target IDs .................')
dti_mapped_target = dti_tids_set
targets_mapped_intersect = print_comparison("Targets (tid)", dti_mapped_target, comp_dti_targets, output=True)

dti_mapped_pairs = DTIs_set 
pairs_mapped_intersect = print_comparison("Drug-target pairs", dti_mapped_pairs, comp_dti_drug_pairs, output=True)


all_intersect_comp.append(['drug_mechanisms mapped', 0, mol_intersect, targets_mapped_intersect, 0, 0,  pairs_mapped_intersect])

------
Drugs (molregno)
current:        3528
comparison:     686
intersection:   629
................. Before mapping target IDs .................
------
Targets (tid)
current:        1080
comparison:     908
intersection:   654
------
Drug-target pairs
current:        4535
comparison:     1221
intersection:   653
................. Including mapping target IDs .................
------
Targets (tid)
current:        1369
comparison:     908
intersection:   895
------
Drug-target pairs
current:        8579
comparison:     1221
intersection:   1113


# DTI classification

Identify which TIDs are drug targets from the drug_mechanism table and add field called "defined_DTI": 
    Value: "True" if it is a drug with a curated drug_meachnism and "False" if not.

Map again to the drug mechanism table via "tid" to identify therapeutic targets and add column "therapeutic_target":
    Value: "True" | "False"

Use this logic to define per compound/target pair whether it corresponds to:
    
    drug and its therapeutic target "DTI": "D_DT";
        
    a drug target but not a drug "DTI": "DT";
        
    not a drug and not a drug target "DTI": "NDT";
        
(done by first creating a new column "DT_assoc" in both the master and the mapping table which reflects the molregno-tid association and then mapping it)

In [29]:
drugs_set = set(df_combined[df_combined['max_phase'] == 4]["parent_molregno"])

In [30]:
df_combined['therapeutic_target'] = df_combined['tid'].isin(dti_tids_set)
df_combined['is_drug'] = df_combined['parent_molregno'].isin(drugs_set)
df_combined['DT_assoc'] = df_combined.agg('{0[parent_molregno]}_{0[tid]}'.format, axis=1)

In [31]:
df_combined['DTI'] = "Nan"
df_combined.loc[df_combined['DT_assoc'].isin(DTIs_set), 'DTI'] = "D_DT"
df_combined.loc[(df_combined['therapeutic_target']== True) & (df_combined['is_drug']== False), 'DTI'] = "DT"
df_combined.loc[(df_combined['therapeutic_target']== False) & (df_combined['is_drug']== False), 'DTI'] = "NDT"

In [32]:
############### TESTING: before reducing to D_DT and DT ###############
add_intersections(df_combined, "pre DTI")

In [33]:
# keep only D_DT and DT
df_combined = df_combined[(df_combined['DTI'].isin(['D_DT', 'DT']))]

In [34]:
############### TESTING: after reducing to D_DT and DT ###############
add_intersections(df_combined, "post DTI")

In [35]:
############### TESTING ###############
# This was supposedly changed to include single proteins for protein families / complexes
df_combined[(df_combined['compound_pref_name'] == "PRAZOSIN") 
            &(df_combined['Target_pref_name'].str.contains("drenergic receptor"))  
            & (df_combined['DTI'] == 'D_DT')
            & (df_combined['only_binding'] == True)][['tid', 'pchembl_value_mean', 'pchembl_value_median', 'pchembl_value_max', 'target_type']]

,tid,pchembl_value_mean,pchembl_value_median,pchembl_value_max,target_type
858,125,9.518750,9.490,10.40,SINGLE PROTEIN
859,128,9.283636,9.390,9.68,SINGLE PROTEIN
860,103,9.430000,9.485,9.70,SINGLE PROTEIN
861,104723,9.502000,9.700,10.27,PROTEIN FAMILY


In [36]:
############### TESTING ###############
# This potentially doesn't make sense: the max pcheml should be the same as the protein family?
df_comparison[(df_comparison['CMPD_PREF_NAME'] == "PRAZOSIN") 
            &(df_comparison['TARGET_PREF_NAME'].str.contains("drenergic receptor"))  
            & (df_comparison['DTI'] == 'D_DT')][['TID', 'PCHEMBL_MEAN', 'PCHEMBL_MEDIAN', 'PCHEMBL_MAX', 'TARGET_TYPE']]

,TID,PCHEMBL_MEAN,PCHEMBL_MEDIAN,PCHEMBL_MAX,TARGET_TYPE
65993,128,9.28,9.39,9.68,SINGLE PROTEIN
250309,103,9.43,9.49,9.70,SINGLE PROTEIN
280827,125,9.52,9.49,10.40,SINGLE PROTEIN
493849,104723,9.50,9.70,10.27,PROTEIN FAMILY


# Add scaffold smiles

In [37]:
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold

# note: this takes a few minutes to calculate for all molecules
def calculate_scaffolds(smiles_set):
    scaffolds_dict = dict()
    scaffolds_no_stereo_dict = dict()
    for smiles in tqdm(smiles_set):
        mol = Chem.MolFromSmiles(smiles)
        if Chem.rdMolDescriptors.CalcNumRings(mol) == 0:
            continue

        scaffold = MurckoScaffold.GetScaffoldForMol(mol)
        scaffolds_dict[smiles] = Chem.MolToSmiles(scaffold)
        
        # repeat after removing stereochemistry
        Chem.RemoveStereochemistry(mol)
        scaffold_no_stereo = MurckoScaffold.GetScaffoldForMol(mol)
        scaffolds_no_stereo_dict[smiles] = Chem.MolToSmiles(scaffold_no_stereo)
        
    return scaffolds_dict, scaffolds_no_stereo_dict

In [38]:
# smiles_set = set(df_combined["canonical_smiles"])
# scaffolds_dict, scaffolds_no_stereo_dict = calculate_scaffolds(smiles_set)

# df_combined['scaffold_wo_stereo'] = df_combined['canonical_smiles'].map(scaffolds_no_stereo_dict)
# df_combined["scaffold_w_stereo"] = df_combined['canonical_smiles'].map(scaffolds_dict)

# Filter for targets (all assay types) with at least 100 comparator compounds

In [39]:
# consider all assay types
df_combined_all_assays = df_combined[(df_combined['only_binding'] == False)]

In [40]:
min_nof_cpds = 100

df_rest = df_combined_all_assays[df_combined_all_assays['max_phase'] < 4]
comparator_counts = df_rest.groupby(['tid_mutation'])['parent_molregno'].count()
targets_w_enough_cpds = comparator_counts[comparator_counts >= min_nof_cpds].index.tolist()
df_limited_targets = df_combined_all_assays.query('tid_mutation in @targets_w_enough_cpds')

In [41]:
# df_limited_targets.to_csv(path_results+"ChEMBL"+chembl_version+"_DTI_all_assays_100.csv", sep = ";")

In [42]:
############### TESTING ###############
print("Targets with >= 100 comparators:", len(targets_w_enough_cpds), "\n")  

for phase in range(1, 5):
    print("Phase: ", phase)
    df_phase = df_limited_targets[df_limited_targets['max_phase'] == phase]
    print(df_phase.DTI.value_counts())
    print("Unique targets with annotated cpds in phase " + str(phase) + ":", df_phase.tid.nunique())
    print()

Targets with >= 100 comparators: 563 

Phase:  1
DT    1184
Name: DTI, dtype: int64
Unique targets with annotated cpds in phase 1: 293

Phase:  2
DT    2175
Name: DTI, dtype: int64
Unique targets with annotated cpds in phase 2: 406

Phase:  3
DT    1781
Name: DTI, dtype: int64
Unique targets with annotated cpds in phase 3: 402

Phase:  4
D_DT    1146
Name: DTI, dtype: int64
Unique targets with annotated cpds in phase 4: 286



In [43]:
############### TESTING: all assay types ###############
add_intersections(df_combined_all_assays, "all assays")
add_intersections(df_limited_targets, "all, >= 100")

# Filter for targets (only binding assays) with at least 100 comparator compounds

In [44]:
# consider only binding assays and therapeutic targets
df_comb_B = df_combined[(df_combined['only_binding'] == True)]

In [45]:
min_nof_cpds = 100

df_rest_B = df_comb_B[df_comb_B['max_phase'] < 4]
comparator_counts_B = df_rest_B.groupby(['tid_mutation'])['parent_molregno'].count()
targets_w_enough_cpds_B = comparator_counts_B[comparator_counts_B >= min_nof_cpds].index.tolist()
df_limited_targets_B = df_comb_B.query('tid_mutation in @targets_w_enough_cpds_B')

In [46]:
# df_limited_targets_B.to_csv(path_results+"ChEMBL"+chembl_version+"_DTI_binding_assays_100.csv", sep = ";")

In [47]:
############### TESTING ###############
print("Targets with >= 100 comparators:", len(targets_w_enough_cpds_B), "\n")  

for phase in range(1, 5):
    print("Phase: ", phase)
    df_phase = df_limited_targets_B[df_limited_targets_B['max_phase'] == phase]
    print(df_phase.DTI.value_counts())
    print("unique targets with annotated cpds in phase " + str(phase) + ":", df_phase.tid_mutation.nunique())
    print()

Targets with >= 100 comparators: 534 

Phase:  1
DT    812
Name: DTI, dtype: int64
unique targets with annotated cpds in phase 1: 274

Phase:  2
DT    1760
Name: DTI, dtype: int64
unique targets with annotated cpds in phase 2: 381

Phase:  3
DT    1466
Name: DTI, dtype: int64
unique targets with annotated cpds in phase 3: 380

Phase:  4
D_DT    1090
Name: DTI, dtype: int64
unique targets with annotated cpds in phase 4: 279



In [48]:
############### TESTING: binding assays ###############
add_intersections(df_comb_B, "binding")
add_intersections(df_limited_targets_B, "b, >= 100")

# Overview of comparisons to old ChEMBL26 dataset

In [49]:
############### TESTING: deleopment of intersection(curr_data, old_dataset) ###############
print("intersection(curr_data, old_dataset)")
pd.DataFrame(all_intersect_comp,
                   columns=['type', 'mols', 'drugs', 'targets', 'drug_targets', 'cpd_target', 'drug_target'])

intersection(curr_data, old_dataset)


,type,mols,drugs,targets,drug_targets,cpd_target,drug_target
0,comparison,378661,686,910,351,561772,1221
1,init,378661,616,907,349,560759,1120
2,cpd props,378661,616,907,349,560759,1120
3,drug_mechanisms mapped,0,629,895,0,0,1113
4,pre DTI,378661,616,907,349,560759,1120
5,post DTI,375142,595,889,335,551887,1075
6,all assays,375142,595,889,335,551887,1075
7,"all, >= 100",370066,557,548,267,542873,986
8,binding,375116,595,889,335,551886,1075
9,"b, >= 100",369017,554,521,258,541213,975


In [50]:
############### TESTING: deleopment of intersection(curr_data, old_dataset) limited to >= 100 cpds ###############
print("intersection(curr_data, old_dataset) limited to >= 100 cpds")
pd.DataFrame(all_intersect_comp_100,
                   columns=['type', 'mols', 'drugs', 'targets', 'drug_targets', 'cpd_target', 'drug_target'])

intersection(curr_data, old_dataset) limited to >= 100 cpds


,type,mols,drugs,targets,drug_targets,cpd_target,drug_target
0,comparison_100,372275,642,521,269,549818,1102
1,init,372275,572,520,267,548870,1005
2,cpd props,372275,572,520,267,548870,1005
3,pre DTI,372275,572,520,267,548870,1005
4,post DTI,368956,555,515,257,540778,971
5,all assays,368956,555,515,257,540778,971
6,"all, >= 100",368716,554,513,256,540614,970
7,binding,368931,555,515,257,540777,971
8,"b, >= 100",368676,554,513,256,540613,970


In [51]:
############### TESTING: deleopment of size(curr_data) vs. size(old_dataset) ###############
print("Size(curr_data) vs. Size(old_dataset)")
pd.DataFrame(all_length_comp,
                   columns=['type', 'mols', 'drugs', 'targets', 'drug_targets', 'cpd_target', 'drug_target'])

Size(curr_data) vs. Size(old_dataset)


,type,mols,drugs,targets,drug_targets,cpd_target,drug_target
0,comparison,378661,686,910,351,561772,1221
1,comparison_100,372275,642,521,269,549818,1102
2,init,923862,1637,6893,2569,2094029,21601
3,cpd props,923862,1637,6893,2569,2094029,21601
4,pre DTI,922987,1623,6887,2568,2091609,21493
5,post DTI,479110,691,1349,481,720205,1404
6,all assays,479110,691,1349,481,720205,1404
7,"all, >= 100",472807,649,563,293,706646,1146
8,binding,383867,656,1305,467,569726,1347
9,"b, >= 100",377058,612,534,279,555590,1090


## Analyse differences in pchembl_mean

In [52]:
def round_decimal(x):
    return str(round(x+10**(-len(str(x))-1), 2))

def identify_mean_differences(data):
    problems = 0
    targets_with_issues = set()
    for index, row in data.iterrows():
        now_mean = round_decimal(float(row['pchembl_value_mean']))
        comp_mean = round_decimal(float(row['PCHEMBL_MEAN']))
        if now_mean != comp_mean:
            problems += 1
            targets_with_issues.add(row['tid_mutation'])

    print("Rows:", len(data))
    print("Mean differences:", problems)
    print("#Targets:", len(set(data['tid_mutation'])))
    print("#Targets with differences:", len(targets_with_issues))

In [53]:
a = df_limited_targets_B[['parent_molregno', 'tid', 'tid_mutation', 'pchembl_value_mean', 'pchembl_value_max', 'pchembl_value_median']].copy()
a['PAIRS'] = a.agg('{0[parent_molregno]}_{0[tid_mutation]}'.format, axis=1)
b = df_comparison[['PCHEMBL_MEAN', 'PCHEMBL_MAX', 'PCHEMBL_MEDIAN', 'PAIRS']].copy()
overlapping_data = a.merge(b, on='PAIRS', how = 'inner')
overlapping_data

,parent_molregno,tid,tid_mutation,pchembl_value_mean,pchembl_value_max,pchembl_value_median,PAIRS,PCHEMBL_MEAN,PCHEMBL_MAX,PCHEMBL_MEDIAN
0,933,10193,10193,7.3400,8.22,7.490,933_10193,7.34,8.22,7.49
1,933,15,15,8.1625,8.70,8.305,933_15,8.16,8.70,8.31
2,933,11063,11063,7.8900,7.89,7.890,933_11063,7.89,7.89,7.89
3,933,12209,12209,7.0700,7.07,7.070,933_12209,7.07,7.07,7.07
4,933,11042,11042,7.4700,7.47,7.470,933_11042,7.47,7.47,7.47
...,...,...,...,...,...,...,...,...,...,...
541208,2334567,12687,12687,6.2500,6.25,6.250,2334567_12687,6.25,6.25,6.25
541209,2334133,12687,12687,6.7500,6.75,6.750,2334133_12687,6.75,6.75,6.75
541210,2326230,12687,12687,6.6800,6.68,6.680,2326230_12687,6.68,6.68,6.68
541211,2333281,12687,12687,6.1400,6.14,6.140,2333281_12687,6.14,6.14,6.14


In [54]:
identify_mean_differences(overlapping_data)

Rows: 541213
Mean differences: 2074
#Targets: 521
#Targets with differences: 311


In [55]:
a = df_comb_B[df_comb_B["max_phase"] == 4].copy()
a['PAIRS'] = a.agg('{0[parent_molregno]}_{0[tid_mutation]}'.format, axis=1)
now_drug_pairs = set(a['PAIRS'])
comp_drug_pairs = set(df_comparison[df_comparison['DTI'] == 'D_DT']["PAIRS"])
intersect = now_drug_pairs.intersection(comp_drug_pairs)
overlapping_data = a.merge(df_comparison[df_comparison['DTI'] == 'D_DT'], on='PAIRS', how = 'inner')
limited_overlapping = overlapping_data[overlapping_data['PAIRS'].isin(intersect)]
identify_mean_differences(limited_overlapping)

Rows: 1075
Mean differences: 46
#Targets: 332
#Targets with differences: 36
